# Задание 1.

In [576]:
from string import punctuation
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import Counter

In [577]:
dostoevsky = open('besy_dostoevsky.short').read()

In [578]:
def normalize(text):
    normalized_text = [
        word.strip(punctuation) for word in text.lower().split()
    ]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [579]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0, len(tokens) - n+1):
        ngrams.append(' '.join(tokens[i: i+n]))
    return ngrams

In [580]:
sentences_dostoevsky = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]

In [581]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence, 2))
    trigrams_dostoevsky.update(ngrammer(sentence, 3))

In [582]:
trigrams_dostoevsky.most_common(10)

[('<start> <start> —', 1930),
 ('<start> <start> я', 283),
 ('<start> <start> он', 187),
 ('<start> <start> но', 181),
 ('<start> — я', 133),
 ('<start> <start> и', 102),
 ('<start> <start> а', 100),
 ('<start> <start> в', 84),
 ('<start> — да', 77),
 ('<start> <start> она', 76)]

In [583]:
bigrams_dostoevsky.most_common(10)

[('<start> <start>', 6142),
 ('<start> —', 1930),
 ('<start> я', 283),
 ('варвара петровна', 211),
 ('николай всеволодович', 198),
 ('<start> он', 187),
 ('— я', 184),
 ('и не', 182),
 ('<start> но', 181),
 ('степан трофимович', 180)]

In [584]:
matrix_dostoevsky = np.zeros((
    len(bigrams_dostoevsky),
    len(unigrams_dostoevsky),
))

In [585]:
id2word_dostoevsky = list(unigrams_dostoevsky)
id2word_bigrams_dostoevsky = list(bigrams_dostoevsky)

word2id_dostoevsky = {word: i for i, word in enumerate(id2word_dostoevsky)}
word2id_bigrams_dostoevsky = {word: i for i, word in enumerate(id2word_bigrams_dostoevsky)}

In [586]:
for ngram in trigrams_dostoevsky:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_dostoevsky[
        word2id_bigrams_dostoevsky[bigram]
    ][
        word2id_dostoevsky[word3]
    ] = (
        trigrams_dostoevsky[ngram] / bigrams_dostoevsky[bigram]
    )

In [587]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100):
    text = []
    
    token1 = token2 = word2id['<start>']
    
    text.append(id2word[token1])
    text.append(id2word[token2])
    for i in range(n):
        try:
            token3 = np.random.choice(
                matrix.shape[1],
                p=matrix[
                    bigram2id[
                        '{} {}'.format(
                            id2word[token1],
                            id2word[token2],
                        )
                    ]
                ]
            )
            text.append(id2word[token3])
            if id2word[token3] == '<end>':
                token1 = token2 = word2id['<start>']
                text.append(id2word[token1])
                text.append(id2word[token2])
            else:
                token1 = token2
                token2 = token3
        except ValueError:
            pass
        
    return ' '.join(text)

In [597]:
print(
    generate(
        matrix_dostoevsky,
        id2word_dostoevsky,
        word2id_dostoevsky,
        id2word_bigrams_dostoevsky,
        word2id_bigrams_dostoevsky,
    ).replace(' <end>', '\n').replace('<start> ', '\n')
)



— твердо заявил николай всеволодович оставшись один начнет колотить кулаками в дверь с неестественною силой оторвал от оконца в дверях как раз пьяный в пруду утонул к тому же ты мне только самый самый маленький срок
 

— он это про головы сам выдумал из книги и колокола и церковные дела потому я был так сказать даже хозяйственного такта которыми она обладала все-таки в ее православие… я верую в россию своим управляющим чтоб от сей поры обращаться с крестьянами как можно небрежнее выпустил николай всеволодович долго пробирался около заборов не отдаляясь от берега но твердо находя свою дорогу и даже


# Задание 2.



In [598]:
from nltk import sent_tokenize
from gensim.models.phrases import Phrases

In [599]:
text = open('anna_karenina.txt').read()

In [600]:
sents = sent_tokenize(text)
sents[100:110]

['И вот возьми телеграмму, передай, что они скажут.',
 '«Попробовать хотите», – понял Матвей, но он сказал только:\n\n– Слушаю-с.',
 'Степан Аркадьич уже был умыт и расчесан и сбирался одеваться, когда Матвей, медленно ступая поскрипывающими сапогами, с телеграммой в руке, вернулся в комнату.',
 'Цирюльника уже не было.',
 '– Дарья Александровна приказали доложить, что они уезжают.',
 'Пускай делают, как им, вам то есть, угодно, – сказал он, смеясь только глазами, и, положив руки в карманы и склонив голову набок, уставился на барина.',
 'Степан Аркадьич помолчал.',
 'Потом добрая и несколько жалкая улыбка показалась на его красивом лице.',
 '– А?',
 'Матвей?']

In [601]:
symbols = [list(sent) for sent in sents]

In [602]:
phrases1 = Phrases(symbols, scoring='npmi', threshold=-1)

In [603]:
phrases2 = Phrases(phrases1[symbols], scoring='npmi', threshold=-1)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [604]:
phrases3 = Phrases(phrases2[phrases1[symbols]], scoring='npmi', threshold=-1)

In [606]:
phrases4 = Phrases(phrases3[phrases2[phrases1[symbols]]], scoring='npmi', threshold=-1)

In [607]:
phrases5 = Phrases(phrases4[phrases3[phrases2[phrases1[symbols]]]], scoring='npmi', threshold=-1)

In [608]:
phrases = Phrases(phrases5[phrases4[phrases3[phrases2[phrases1[symbols]]]]], scoring='npmi', threshold=-1)

In [609]:
cmn = Counter(phrases.vocab).most_common(15)

In [612]:
list(phrases[phrases5[phrases4[phrases3[phrases2[phrases1[symbols[:100]]]]]]])

[['A',
  'n_n',
  'o_t',
  'a_t',
  'i_o',
  'n',
  '\n_\n_\n_«',
  'А_н_н_а',
  ' _К_а_р_е_н_и_н',
  'а_»',
  ' _–_ _э',
  'т_о_ _с',
  'л_о_ж_н',
  'о_е_,_ ',
  'п_с',
  'и_х',
  'о_л_о_г',
  'и_ч_е_с',
  'к_и_ _у',
  'т_о',
  'н_ч_е_н',
  'н_о_е_,',
  ' _о_с_т',
  'р_о',
  'п_р_о_б',
  'л_е',
  'м_н_о_е',
  ' _п_р_о_и_з_в_е',
  'д_е_н_и',
  'е_,_ _н',
  'а_с',
  'ы',
  'щ_е_н_н',
  'о_е_ _п',
  'р_и_м_е',
  'т_а_м_и',
  ' _в_р_е_м_е_н_и_.'],
 ['Л', '.', 'Н', '.'],
 ['Т_о',
  'л_с',
  'т_о_й_ ',
  'н_а_ _с',
  'т_р_а_н',
  'и_ц_а_х',
  ' _п_р_о_и_з_в_е',
  'д_е_н_и',
  'я_ _п_о',
  'к_а_з_ы_в_а_е_т',
  ',_ _к_а',
  'к_ _р_у',
  'ш_а',
  'т_с_я_ ',
  'о_с_т_а',
  'т_к_и_ ',
  'п_а_т_р',
  'и_а',
  'р_х',
  'а_л_ь_н_о_г_о_ ',
  'у_к_л_а',
  'д_а_ _ж',
  'и_з_н_и',
  ' _в_ _Р_о_с_с_и',
  'и_ _п_о',
  'д_ _н_а',
  'т_и',
  'с_к_о_м',
  ' _б_у_р',
  'ж_у',
  'а_з_н_о',
  'г_о_ _п',
  'р_о_г_р',
  'е_с_с_а',
  ',_ _к_а',
  'к_ _п_а',
  'д_а_ю_т',
  ' _н_р_а',
  'в_ы_,_ ',
  'о_с_л_а',
  'б

# Итог
Что-то получилось. Возможно, если прогнать ещё несколько раз, будет совсем хорошо.